In [2]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-28 18:24:11,317.317 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-28 18:24:11,317.317 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [3]:
#%pip install pyEX

In [4]:
#%pip install awswrangler

In [5]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'pk_79d147436c1349f3abbec37591323e52
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

/usr/local/lib/python3.7/site-packages/tzlocal/unix.py:177: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


In [15]:
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        print('panda UDF')
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'close': None, 
                              'open': None, 
                              'symbol': local_sym}])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','close','open','symbol']
        self.secrets = {
            'IEX':{},
            'alpaca':{ 'token':'alpaca_marketdata_key',
                      'secret':'alpaca_marketdata_secret'}
                       }
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp, close double, open double, symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
            
    def get_secret(self,secret_name,region_name):
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            # For a list of exceptions thrown, see
            # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
            raise e

        # Decrypts secret using the associated KMS key.
        secret = json.loads(get_secret_value_response['SecretString'])[secret_name]
        return(secret)
        

mdg=MarketDataGateway()
mdg.set_highwatermarks()
# set bc vars
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
iex_token_bc =sc.broadcast(mdg.iex_token)
iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
# set bc vars
mdg.set_new_symbol_df()
mdg.write_new_symbol_df()
print(mdg.write_stats.toPandas().sort_values(['symbol']))
mdg.rewrite_if_dups()

   symbol  data_count
2    ARKG           8
7    ARKW           8
5    BITQ           8
6    BLCN           9
18   BTBT           9
29   COPX           9
34   DAPP           9
12   DRIV           9
36    EWZ           9
40   FIVG           9
41   FTXG           8
17    GNR           8
8    HACK           8
37    HYG           9
30    IGV           9
1     IWN           9
20   JETS           8
26   KWEB           8
4     LIT           9
38   MLPA           9
35   MSTR           9
27    PPA           9
10   REET           9
25   RIOT           8
39   SCHH           9
14   SKYY           9
42   SPBC           9
13   SRVR           8
11    URA           8
9     UUP           8
15    VUG           9
16    XAR           8
33    XBI           9
19    XLB           9
31    XLE           9
28    XLF           9
0     XLI           8
22    XLP           9
24   XLRE           9
32    XLV           9
21    XLY           8
23    XOP           9
3     XSD           8
no dups


In [16]:
mdg.old_df_after_write.distinct().groupby('date','symbol').count().where('count>1').count()#.orderBy(['count','symbol'],ascending=False).show()

0

In [17]:
mdg.highwatermark_pd

,symbol,max_date,min_date,date_length
0,SPY,2023-01-27,2006-12-29,5873 days
1,XSD,2023-01-17,2007-03-01,5801 days
2,FXE,2023-01-27,2007-03-02,5810 days
3,SOXX,2023-01-17,2007-03-01,5801 days
4,FDN,2023-01-27,2006-12-29,5873 days
...,...,...,...,...
141,BITF,2023-01-13,2021-06-21,571 days
142,DAPP,2023-01-13,2021-04-14,639 days
143,HUT,2023-01-27,2021-06-15,591 days
144,FIVG,2023-01-13,2019-03-05,1410 days


In [18]:
#lpd=mdg.highwatermark_pd
#sym='abc'
#sym_pd = lpd.query(f"symbol=='{sym}'")['max_date']
#if len(sym_pd)>0:
#    max_date = lpd.query(f"symbol=='{sym}'")['max_date'].iloc[0]
#else:
#    max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
#last_days=(datetime.datetime.now()-max_date).days
#res_pd = pyEX_cl.chartDF(symbol=sym, timeframe=timeframe,last=last_days).query(f"date>'{(max_date)}'")
res_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")

NameError: name 'res_pd' is not defined

In [ ]:
max_date.strftime("%Y-%m-%d")

In [ ]:

#base_url = 'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
#headers = {
#      'content-type': "application/json"
#  }

res = requests.get(base_url)
pd.DataFrame(json.loads(res.text))


In [ ]:
(max_date-10).strftime('%Y-%m-%d')

In [ ]:
lpd